In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import pickle as pkl
import torch.nn.functional as F
import torchvision
from attack import attack, test_model,parse_param


In [2]:
import re
def parse_param(param):
    reg = re.compile("\.\d+\.")
    finded = reg.findall(param)
    if len(finded) == 0:
        pass
    else:
        for f in finded:
            f = f[1:-1]
            param = param.replace(f".{f}.", f"[{f}].")
    return param

In [3]:
from datasets import load_cifar10, load_cifar100
from models.resnet import load_cifar10_resnet50, load_cifar100_resnet50
model = load_cifar10_resnet50()


In [4]:
all_param_names = list()
for name, param in model.named_parameters():
    all_param_names.append(name)

In [5]:
train_loaders, test_dataloaders,train_dataloader_all, test_dataloader_all = load_cifar10()
all_totals = list()
all_totals.append(attack(train_dataloader_all, all_param_names, load_cifar10_resnet50, alpha=0.0001))


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 196/196 [01:14<00:00,  2.64it/s]


0.0005741861485689878


100%|██████████| 196/196 [01:14<00:00,  2.62it/s]


0.4984813861083984


...

100%|██████████| 196/196 [01:19<00:00,  2.46it/s]


60.13210147460938


100%|██████████| 196/196 [01:19<00:00,  2.45it/s]


228.083117734375


d:\Documents\GitHub\Neural-importance\P1模型训练+归因测试\参数归因Cifar10不分label\attack.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  param_totals = np.array(param_totals)
d:\Documents\GitHub\Neural-importance\P1模型训练+归因测试\参数归因Cifar10不分label\utils.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)


In [6]:
all_totals


[{'conv1.weight': array([[[[  4.5820274 ,   1.7457509 ,   1.5391767 ],
           [  4.571298  ,   2.9696498 ,   5.696471  ],
           [  7.8024497 ,   6.800726  ,   7.0965147 ]],
  
          [[  2.408275  ,  -0.5678758 ,  -0.93329114],
           [  4.4862695 ,   1.975326  ,   5.8244514 ],
           [  4.3637404 ,   6.8257995 ,   7.7279553 ]],
  
          [[ -6.2458315 ,  -9.045525  ,  -8.877756  ],
           [ -3.8360531 ,  -4.796874  ,  -2.756482  ],
           [ -6.293932  ,  -3.6749847 ,  -0.22354884]]],
  
  
         [[[ -9.34878   , -17.89032   , -36.111317  ],
           [-13.403136  , -12.48887   , -26.964504  ],
           [ -9.983121  ,  -2.7078698 ,  -7.3233185 ]],
  
          [[ -6.2526593 , -12.095538  , -30.559214  ],
           [-13.490238  , -11.64208   , -24.116909  ],
           [-11.5316725 ,  -2.04692   ,  -6.7143097 ]],
  
          [[ -4.934721  , -11.442536  , -28.534273  ],
           [-11.677617  ,  -8.298397  , -23.355425  ],
           [-12.959196  ,

In [7]:
pkl.dump(all_totals, open("weights/cifar10_resnet50.pkl", "wb"))

In [8]:
thre = 0.4
net = load_cifar10_resnet50()
param_remove = dict()
for param in all_param_names:
    param_remove[param] = None
for i in range(len(all_totals)):
    totals = all_totals[i]
    totals = [totals[param] for param in all_param_names]
    param_weights = [eval("net." + parse_param(param) + ".cpu().detach().numpy()")
                     for param in all_param_names]
    combine = [np.abs(total * weight) for total, weight in zip(totals, param_weights)]
    combine = np.array(combine)
    combine_flatten = np.concatenate([combine_.flatten() for combine_ in combine],axis=0)
    threshold = np.sort(combine_flatten)[::-1][int(len(combine_flatten) * thre)]
    for idx,param in enumerate(all_param_names):
        if param_remove[param] is None:
            param_remove[param] = combine[idx] > threshold
        else:
            t = combine[idx] > threshold
            param_remove[param] = param_remove[param] | t

C:\Users\Zhang\AppData\Local\Temp\ipykernel_28656\3025117405.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  combine = np.array(combine)


In [9]:
temp = 0
all_num = 0
for param in param_remove:
    temp += param_remove[param].sum()
    all_num += param_remove[param].size
    print(param, param_remove[param].mean())

conv1.weight 0.9432870370370371
bn1.weight 1.0
bn1.bias 1.0
layer1.0.conv1.weight 0.75927734375
layer1.0.bn1.weight 0.984375
layer1.0.bn1.bias 0.953125
layer1.0.conv2.weight 0.6896430121527778
layer1.0.bn2.weight 1.0
layer1.0.bn2.bias 0.96875
layer1.0.conv3.weight 0.69451904296875
layer1.0.bn3.weight 0.9375
layer1.0.bn3.bias 0.90625
layer1.0.shortcut.0.weight 0.7081298828125
layer1.0.shortcut.1.weight 0.921875
layer1.0.shortcut.1.bias 0.90625
layer1.1.conv1.weight 0.70184326171875
layer1.1.bn1.weight 0.96875
layer1.1.bn1.bias 0.90625
layer1.1.conv2.weight 0.7052137586805556
layer1.1.bn2.weight 1.0
layer1.1.bn2.bias 0.921875
layer1.1.conv3.weight 0.6483154296875
layer1.1.bn3.weight 0.890625
layer1.1.bn3.bias 0.9375
layer1.2.conv1.weight 0.64678955078125
layer1.2.bn1.weight 0.984375
layer1.2.bn1.bias 0.890625
layer1.2.conv2.weight 0.6098904079861112
layer1.2.bn2.weight 1.0
layer1.2.bn2.bias 0.984375
layer1.2.conv3.weight 0.63165283203125
layer1.2.bn3.weight 0.87109375
layer1.2.bn3.bias 0

In [10]:
temp / all_num

0.39999996598761217

In [11]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    correct, all = test_model(net, test_dataloader_all)
    print("原始准确率", correct / all)


原始准确率 0.954


In [13]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    for param in all_param_names:
        param_ = parse_param(param)
        try:
            exec("net." + param_ + "[~param_remove[param]] = 0")
        except:
            exec("net." + param_ + "[~param_remove[param],:] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("现在准确率", correct / all)


现在准确率 0.8379


In [14]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    for param in all_param_names:
        param_ = parse_param(param)
        keep_rate = param_remove[param].sum() / param_remove[param].size
        weight_flatten = eval("net." + param_ + ".cpu().detach().numpy()").flatten()
        threshold = np.sort(weight_flatten)[int(len(weight_flatten) * (1 - keep_rate))]
        try:
            exec("net." + param_ + "[eval('net.' + param_ + '.cpu().detach().numpy()') < threshold] = 0")
        except:
            exec("net." + param_ + "[eval('net.' + param_ + '.cpu().detach().numpy()') < threshold,:] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("对比试验准确率", correct / all)


对比试验准确率 0.1
